In [1]:
import numpy as np
import pandas as pd
# from icecream import ic
import subprocess
import json
import os
import plotly.graph_objects as go
from matplotlib import pyplot as plt
# import geopandas as gpd
# from shapely.geometry import Polygon
# from shapely.ops import cascaded_union, unary_union
# from shapely import geometry
# from geopy import distance
%matplotlib inline
# import plotly.express as px 
from scipy import signal
from tqdm import tqdm
# from Bio import Phylo
# import ete3
import itertools
import sklearn
from sklearn import metrics

In [7]:
def value_counts_list(lst):
    """
    Computes the frequency count of unique elements in a list and returns a dictionary, sorted by frequency count in
    descending order.

    Args:
    - lst (list): List of elements

    Returns:
    - dict: Dictionary with unique elements as keys and their frequency count as values, sorted by frequency count
    in descending order
    """
    value_counts = {}
    for item in lst:
        if item in value_counts:
            value_counts[item] += 1
        else:
            value_counts[item] = 1
    sorted_value_counts = dict(sorted(value_counts.items(), key=lambda x: x[1], reverse=True))
    return sorted_value_counts

def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [8]:
clinical_biochem = pd.read_csv('../Clinical/TB_Portals_Published_Clinical_Data_August_2023/TB_Portals_Biochemistry_August_2023.csv') #blood test measurements
clinical_DST = pd.read_csv('../Clinical/TB_Portals_Published_Clinical_Data_August_2023/TB_Portals_DST_August_2023.csv') # mostly empty
clinical_patient = pd.read_csv('../Clinical/TB_Portals_Published_Clinical_Data_August_2023/TB_Portals_Patient_Cases_August_2023.csv') # body, background, diagnostic detail
clinical_regimens = pd.read_csv('../Clinical/TB_Portals_Published_Clinical_Data_August_2023/TB_Portals_Regimens_August_2023.csv') #outcome. drug regimen, same patient id number as clinical_patient as cases
clinical_specimen = pd.read_csv('../Clinical/TB_Portals_Published_Clinical_Data_August_2023/TB_Portals_Specimen_August_2023.csv') # collection dates and source - no use

/tmp/ipykernel_31925/730962421.py:2: DtypeWarning: Columns (33,40,41,55,60,61,63,65,66,68,69,71,72,74,82,100,102,106,107,108,109,110,111,113,119,127,128,129,133,135,138,139,140) have mixed types. Specify dtype option on import or set low_memory=False.
  clinical_DST = pd.read_csv('../Clinical/TB_Portals_Published_Clinical_Data_August_2023/TB_Portals_DST_August_2023.csv') # mostly empty
/tmp/ipykernel_31925/730962421.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  clinical_patient = pd.read_csv('../Clinical/TB_Portals_Published_Clinical_Data_August_2023/TB_Portals_Patient_Cases_August_2023.csv') # body, background, diagnostic detail


In [9]:
clinical_patient = clinical_patient.drop(columns=['education', 'image_body_site', 'dissemination', 'lungcavity_size', 'anomaly_of_mediastinum_vessels_develop', 'affect_pleura', 'shadow_pattern', 'affect_level', 'pneumothorax', 'plevritis', 'affected_segments', 'nodicalcinatum', 'process_prevalence', 'thromboembolism_of_the_pulmonaryartery', 'posttbresiduals', 'lung_capacity_decrease', 'bronchial_obstruction', 'anomaly_of_lungdevelop', 'accumulation_of_contrast', 'limfoadenopatia', 'totalcavernum', 'bactec_test', 'le_test', 'hain_test', 'lpaother_test', 'genexpert_test', 'bactec_isoniazid', 'bactec_rifampicin', 'bactec_streptomycin', 'bactec_ethambutol', 'bactec_ofloxacin', 'bactec_capreomycin', 'bactec_amikacin', 'bactec_kanamycin', 'bactec_pyrazinamide', 'bactec_levofloxacin', 'bactec_moxifloxacin', 'bactec_p_aminosalicylic_acid', 'bactec_prothionamide', 'bactec_cycloserine', 'bactec_amoxicillin_clavulanate', 'bactec_mycobutin', 'bactec_delamanid', 'bactec_bedaquiline', 'bactec_imipenem_cilastatin', 'bactec_linezolid', 'bactec_clofazimine', 'bactec_clarithromycin', 'bactec_fluoroquinolones', 'bactec_aminoglycosides_injectible_agents', 'le_isoniazid', 'le_rifampicin', 'le_streptomycin', 'le_ethambutol', 'le_ofloxacin', 'le_capreomycin', 'le_amikacin', 'le_kanamycin', 'le_pyrazinamide', 'le_levofloxacin', 'le_moxifloxacin', 'le_p_aminosalicylic_acid', 'le_prothionamide', 'le_cycloserine', 'le_amoxicillin_clavulanate', 'le_mycobutin', 'le_delamanid', 'le_bedaquiline', 'le_imipenem_cilastatin', 'le_linezolid', 'le_clofazimine', 'le_clarithromycin', 'le_fluoroquinolones', 'le_aminoglycosides_injectible_agents', 'hain_isoniazid', 'hain_rifampicin', 'hain_streptomycin', 'hain_ethambutol', 'hain_ofloxacin', 'hain_capreomycin', 'hain_amikacin', 'hain_kanamycin', 'hain_pyrazinamide', 'hain_levofloxacin', 'hain_moxifloxacin', 'hain_p_aminosalicylic_acid', 'hain_prothionamide', 'hain_cycloserine', 'hain_amoxicillin_clavulanate', 'hain_mycobutin', 'hain_delamanid', 'hain_bedaquiline', 'hain_imipenem_cilastatin', 'hain_linezolid', 'hain_clofazimine', 'hain_clarithromycin', 'hain_fluoroquinolones', 'hain_aminoglycosides_injectible_agents', 'lpaother_isoniazid', 'lpaother_rifampicin', 'lpaother_streptomycin', 'lpaother_ethambutol', 'lpaother_ofloxacin', 'lpaother_capreomycin', 'lpaother_amikacin', 'lpaother_kanamycin', 'lpaother_pyrazinamide', 'lpaother_levofloxacin', 'lpaother_moxifloxacin', 'lpaother_p_aminosalicylic_acid', 'lpaother_prothionamide', 'lpaother_cycloserine', 'lpaother_amoxicillin_clavulanate', 'lpaother_mycobutin', 'lpaother_delamanid', 'lpaother_bedaquiline', 'lpaother_imipenem_cilastatin', 'lpaother_linezolid', 'lpaother_clofazimine', 'lpaother_clarithromycin', 'lpaother_fluoroquinolones', 'lpaother_aminoglycosides_injectible_agents', 'genexpert_isoniazid', 'genexpert_rifampicin', 'truenat_rifampicin', 'stemcell_dose', 'social_risk_factors', 'main_lineage', 'sub_lineage', 'tbp_drug_prediction', 'num_drug_resistant_variants', 'num_other_variants', 'ncbi_sourceorganism', 'ncbi_bioproject', 'ncbi_biosample', 'gene_name', 'qure_bluntedcp', 'qure_abnormal', 'qure_consolidation', 'qure_fibrosis', 'qure_opacity', 'qure_peffusion', 'qure_tuberculosis', 'qure_nodule', 'qure_cavity', 'qure_hilarlymphadenopathy', 'qure_atelectasis', 'uiip_asymmetry_volume', 'uiip_asymmetry_texture', 'uiip_foci_left_lower_sextant', 'uiip_foci_left_middle_sextant', 'uiip_foci_left_upper_sextant', 'uiip_foci_right_lower_sextant', 'uiip_foci_right_middle_sextant', 'uiip_foci_right_upper_sextant', 'uiip_caverns_left_lower_sextant', 'uiip_caverns_left_middle_sextant', 'uiip_caverns_left_upper_sextant', 'uiip_caverns_right_lower_sextant', 'uiip_caverns_right_middle_sextant', 'uiip_caverns_right_upper_sextant']) # drop columns with no data
clinical_regimens = clinical_regimens.drop(columns=['dose','collected','reinfusioned'])
clinical_biochem = clinical_biochem.drop(columns=['Thyroid Stimulating Hormone (miu/l)', 'Hepatitis B Surface Antigen', 'Hepatitis C Virus', 'Free Thyroxine (ng/dl)', 'Glucose: Diabetic Post-Meal (mmol/l)', 'Bilirubin', 'Albumin (g/l)', 'Urea (mmol/l)', 'Alkaline phosphatase (u/l)', 'Lipase (u/l)', 'Activated Partial Thromboplastin Time (seconds)', 'Calcium (mmol/l)', 'International Normalized Ratio', 'Gamma-Glutamyl Transpeptidase (u/l)', 'Sodium (mmol/l)', 'Thrombin Time (seconds)', 'Prothrombin Time (%)', 'Adenosine Deaminase (u/l)', 'C-Reactive Protein (mg/l)', 'Hemoglobin (g/l)', 'Hemoglobin A1c (%)', 'Fasting Blood Sugar (Glucose) (mmol/l)', 'Total Cholesterol (mmol/l)', 'Absolute Neutrophil Count', 'Absolute Lymphocyte Count', 'Absolute Eosinophil Count', 'Neutrophil-lymphocyte ratio'])

In [51]:
clinical_patient_cp = clinical_patient.drop(columns=['patient_id'])
# merged_data = clinical_patient_cp.merge(clinical_regimens, on='condition_id', how='left')
clinical_biochem_cp = clinical_biochem.drop(columns=['patient_id', 'specimen_id', 'observationfhir_id','test_date',	'specimen_collection_site',	'specimen_collection_date'])
merged_data = clinical_patient_cp.merge(clinical_biochem_cp, on='condition_id', how='left')

# merged_data = merged_data[~merged_data['outcome'].isin(['Lost to follow up', 'Still on treatment','Unknown'])]
# merged_data = merged_data.drop(columns = ['identifier','registration_date','case_definition','diagnosis_code','case_definition','diagnosis_code','culturetype','culture'])
merged_data = merged_data.drop(columns = ['period_start','period_end'])
merged_data = merged_data.drop(columns=['specimen', 'x_ray_exists', 'ct_exists', 'genomic_data_exists','uiip_severity_score','rater', 'x_ray_count',  'organization'])

# merged_data.replace('Not Reported', np.nan, inplace=True)

# columns_to_count = ['collapse', 'smallcavities', 'mediumcavities', 'largecavities', 'isanylargecavitybelongtoamultisextantcavity', 'canmultiplecavitiesbeseen', 'infiltrate_lowgroundglassdensity', 'infiltrate_mediumdensity', 'infiltrate_highdensity', 'smallnodules', 'mediumnodules', 'largenodules', 'hugenodules', 'isanycalcifiedorpartiallycalcifiednoduleexist', 'isanynoncalcifiednoduleexist', 'isanyclusterednoduleexists', 'aremultiplenoduleexists', 'lowgroundglassdensityactivefreshnodules', 'mediumdensitystabalizedfibroticnodules', 'highdensitycalcifiedtypicallysequella']
# for column in columns_to_count:
    # merged_data[column] = merged_data[column].apply(lambda x: x.count("Yes") if not pd.isna(x) else x)
merged_data = merged_data.drop(columns=['identifier', 'registration_date','case_definition', 'diagnosis_code','status', 'culture', 'culturetype','microscopy'])

In [52]:
merged_data.columns

Index(['condition_id', 'age_of_onset', 'gender', 'country', 'employment',
       'number_of_children', 'number_of_daily_contacts', 'type_of_resistance',
       'bmi', 'lung_localization', 'overall_percent_of_abnormal_volume',
       'pleural_effusion_percent_of_hemithorax_involved',
       'ispleuraleffusionbilateral', 'other_non_tb_abnormalities',
       'are_mediastinal_lymphnodes_present', 'timika_score', 'collapse',
       'smallcavities', 'mediumcavities', 'largecavities',
       'isanylargecavitybelongtoamultisextantcavity',
       'canmultiplecavitiesbeseen', 'infiltrate_lowgroundglassdensity',
       'infiltrate_mediumdensity', 'infiltrate_highdensity', 'smallnodules',
       'mediumnodules', 'largenodules', 'hugenodules',
       'isanycalcifiedorpartiallycalcifiednoduleexist',
       'isanynoncalcifiednoduleexist', 'isanyclusterednoduleexists',
       'aremultiplenoduleexists', 'lowgroundglassdensityactivefreshnodules',
       'mediumdensitystabalizedfibroticnodules',
       '

In [126]:
merged_data_cp = merged_data.copy()
def process_string(s):
    # Split the string into parts
    if s is np.nan:
        return "no region in the lung"
    parts = s.split(", ")
    # Process each part
    processed_parts = [part.split("-")[0] for part in parts if "-Yes" in part]
    # Check if all items were removed
    if len(processed_parts) == 0:
        return "no region in the lung"
    else:
        return ", ".join(processed_parts)

# List of columns to apply the function
columns_to_process = ['collapse',
       'smallcavities', 'mediumcavities', 'largecavities',
       'isanylargecavitybelongtoamultisextantcavity',
       'canmultiplecavitiesbeseen', 'infiltrate_lowgroundglassdensity',
       'infiltrate_mediumdensity', 'infiltrate_highdensity', 'smallnodules',
       'mediumnodules', 'largenodules', 'hugenodules',
       'isanycalcifiedorpartiallycalcifiednoduleexist',
       'isanynoncalcifiednoduleexist', 'isanyclusterednoduleexists',
       'aremultiplenoduleexists', 'lowgroundglassdensityactivefreshnodules',
       'mediumdensitystabalizedfibroticnodules',
       'highdensitycalcifiedtypicallysequella']

# Apply the function to each specified column
for column in columns_to_process:
    merged_data_cp[column] = merged_data_cp[column].apply(process_string)

# print(df)

# merged_data = merged_data[['condition_id', 'age_of_onset', 'gender', 'country', 'employment',
#        'number_of_children', 'number_of_daily_contacts', 'type_of_resistance',
#        'bmi', 'lung_localization', 'overall_percent_of_abnormal_volume',
#        'pleural_effusion_percent_of_hemithorax_involved',
#        'ispleuraleffusionbilateral', 'other_non_tb_abnormalities',
#        'are_mediastinal_lymphnodes_present', 'timika_score', 'period_span', 'regimen_count',
#        'outcome', 'treatment_status', 'regimen_drug', 'comorbidity',
#        'Total Protein (g/l)', 'Potassium (mmol/l)',
#        'Aspartate Aminotransferase (u/l)', 'Total Bilirubin (umol/l)',
#        'Glucose (mmol/l)', 'Creatinine (umol/l)',
#        'Alanine Aminotransferase (u/l)',
#        'Erythrocyte Sedimentation Rate (mm/hr)']]

# merged_data = pd.concat([merged_data, merged_data_cp], axis = 1)
merged_data_cp.fillna('no information', inplace=True)

In [83]:
merged_data_cp = merged_data_cp.astype(str)

In [57]:
columns = [
'condition_id', 'age_of_onset', 'gender', 'country', 'employment',
       'number_of_children', 'number_of_daily_contacts', 'type_of_resistance',
       'bmi', 'lung_localization', 'overall_percent_of_abnormal_volume',
       'pleural_effusion_percent_of_hemithorax_involved',
       'ispleuraleffusionbilateral', 'other_non_tb_abnormalities',
       'are_mediastinal_lymphnodes_present', 'timika_score', 'collapse',
       'smallcavities', 'mediumcavities', 'largecavities',
       'isanylargecavitybelongtoamultisextantcavity',
       'canmultiplecavitiesbeseen', 'infiltrate_lowgroundglassdensity',
       'infiltrate_mediumdensity', 'infiltrate_highdensity', 'smallnodules',
       'mediumnodules', 'largenodules', 'hugenodules',
       'isanycalcifiedorpartiallycalcifiednoduleexist',
       'isanynoncalcifiednoduleexist', 'isanyclusterednoduleexists',
       'aremultiplenoduleexists', 'lowgroundglassdensityactivefreshnodules',
       'mediumdensitystabalizedfibroticnodules',
       'highdensitycalcifiedtypicallysequella', 'period_span', 'regimen_count',
       'outcome', 'treatment_status', 'regimen_drug', 'comorbidity',
       'Total Protein (g/l)', 'Potassium (mmol/l)',
       'Aspartate Aminotransferase (u/l)', 'Total Bilirubin (umol/l)',
       'Glucose (mmol/l)', 'Creatinine (umol/l)',
       'Alanine Aminotransferase (u/l)',
       'Erythrocyte Sedimentation Rate (mm/hr)'
]
# Convert each item into the desired format.
formatted_columns = [f"{column} = x['{column}']" for column in columns]

# Joining the formatted strings into a single string with line breaks for readability.
formatted_string = "\n".join(formatted_columns)

In [124]:
word_data = []
for i, x in merged_data.iterrows():
    condition_id = x['condition_id']
    age_of_onset = x['age_of_onset']
    
    gender = x['gender']
    country = x['country']
    employment = x['employment']
    number_of_children = x['number_of_children']
    number_of_daily_contacts = x['number_of_daily_contacts']
    type_of_resistance = x['type_of_resistance']
    bmi = x['bmi']
    lung_localization = x['lung_localization']
    overall_percent_of_abnormal_volume = x['overall_percent_of_abnormal_volume']
    pleural_effusion_percent_of_hemithorax_involved = x['pleural_effusion_percent_of_hemithorax_involved']
    ispleuraleffusionbilateral = x['ispleuraleffusionbilateral']
    other_non_tb_abnormalities = x['other_non_tb_abnormalities']
    are_mediastinal_lymphnodes_present = x['are_mediastinal_lymphnodes_present']
    timika_score = x['timika_score']
    collapse = x['collapse']
    smallcavities = x['smallcavities']
    mediumcavities = x['mediumcavities']
    largecavities = x['largecavities']
    isanylargecavitybelongtoamultisextantcavity = x['isanylargecavitybelongtoamultisextantcavity']
    canmultiplecavitiesbeseen = x['canmultiplecavitiesbeseen']
    infiltrate_lowgroundglassdensity = x['infiltrate_lowgroundglassdensity']
    infiltrate_mediumdensity = x['infiltrate_mediumdensity']
    infiltrate_highdensity = x['infiltrate_highdensity']
    smallnodules = x['smallnodules']
    mediumnodules = x['mediumnodules']
    largenodules = x['largenodules']
    hugenodules = x['hugenodules']
    isanycalcifiedorpartiallycalcifiednoduleexist = x['isanycalcifiedorpartiallycalcifiednoduleexist']
    isanynoncalcifiednoduleexist = x['isanynoncalcifiednoduleexist']
    isanyclusterednoduleexists = x['isanyclusterednoduleexists']
    aremultiplenoduleexists = x['aremultiplenoduleexists']
    lowgroundglassdensityactivefreshnodules = x['lowgroundglassdensityactivefreshnodules']
    mediumdensitystabalizedfibroticnodules = x['mediumdensitystabalizedfibroticnodules']
    highdensitycalcifiedtypicallysequella = x['highdensitycalcifiedtypicallysequella']
    period_span = x['period_span']
    regimen_count = x['regimen_count']
    
    outcome = x['outcome'] #!label
    
    treatment_status = x['treatment_status']
    regimen_drug = x['regimen_drug']
    comorbidity = x['comorbidity']
    Total_Protein = x['Total Protein (g/l)']
    Potassium = x['Potassium (mmol/l)']
    Aspartate_Aminotransferase = x['Aspartate Aminotransferase (u/l)']
    Total_Bilirubin = x['Total Bilirubin (umol/l)']
    Glucose = x['Glucose (mmol/l)']
    Creatinine = x['Creatinine (umol/l)']
    Alanine_Aminotransferase = x['Alanine Aminotransferase (u/l)']
    Erythrocyte_Sedimentation_Rate = x['Erythrocyte Sedimentation Rate (mm/hr)']

    formated_input = f"The patient is {gender}, born in {country} and currently employed as {employment}, diagnosed with TB at the age of {age_of_onset}, presents a case with {type_of_resistance} resistance. The patient has {number_of_children} children and interact with {number_of_daily_contacts} individuals daily. \
The patient’s BMI is recorded at {bmi}, and TB primarily affects the {lung_localization} part of the lung(s), with {overall_percent_of_abnormal_volume}% of lung volume showing abnormalities. Pleural effusion involves {pleural_effusion_percent_of_hemithorax_involved}% of the hemithorax, and bilateral effusion is {ispleuraleffusionbilateral}. \
Additional non-TB abnormalities include {other_non_tb_abnormalities}, and mediastinal lymph nodes presence is {are_mediastinal_lymphnodes_present}. The Timika score is {timika_score}, with noted lung collapse {collapse}, and cavity sizes categorized as small ({smallcavities}), medium ({mediumcavities}), and large ({largecavities}), \
with {isanylargecavitybelongtoamultisextantcavity} large cavities being part of a multisextant cavity and multiple cavities visible {canmultiplecavitiesbeseen}. Lung infiltrates are observed with low ({infiltrate_lowgroundglassdensity}), medium ({infiltrate_mediumdensity}), \
and high ({infiltrate_highdensity}) density alongside nodules of various sizes: small ({smallnodules}), medium ({mediumnodules}), large ({largenodules}), and huge ({hugenodules}). \
Calcified or partially calcified nodules exist: {isanycalcifiedorpartiallycalcifiednoduleexist}, with non-calcified {isanynoncalcifiednoduleexist} and clustered {isanyclusterednoduleexists} nodules noted. Multiple nodules are present {aremultiplenoduleexists}, \
with nodules' characteristics detailed as low-ground glass density active/fresh ({lowgroundglassdensityactivefreshnodules}), medium density stabilized/fibrotic ({mediumdensitystabalizedfibroticnodules}), and high density calcified/typical sequela ({highdensitycalcifiedtypicallysequella}). \
Over a period of {period_span} days, the patient has undergone {regimen_count} treatment regimens with the current status being {treatment_status} and receiving {regimen_drug}. Comorbidities include {comorbidity}. Laboratory results are as follows: Total Protein {Total_Protein} g/l, \
Potassium {Potassium} mmol/l, Aspartate Aminotransferase {Aspartate_Aminotransferase} u/l, Total Bilirubin {Total_Bilirubin} umol/l, Glucose {Glucose} mmol/l, Creatinine {Creatinine} umol/l, Alanine Aminotransferase {Alanine_Aminotransferase} u/l, and Erythrocyte Sedimentation Rate {Erythrocyte_Sedimentation_Rate} mm/hr."
    formated = [formated_input, outcome]
    word_data.append(formated)
    
df = pd.DataFrame(word_data, columns=['description', 'outcome'])


In [125]:
df

,description,outcome
0,"The patient is Male, born in Kazakhstan and cu...",Completed
1,"The patient is Male, born in Georgia and curre...",Cured
2,"The patient is Male, born in Georgia and curre...",Cured
3,"The patient is Female, born in Georgia and cur...",Cured
4,"The patient is Male, born in Georgia and curre...",Cured
...,...,...
13673,"The patient is Male, born in Kazakhstan and cu...",Still on treatment
13674,"The patient is Male, born in Ukraine and curre...",Lost to follow up
13675,"The patient is Male, born in Ukraine and curre...",Cured
13676,"The patient is Male, born in Ukraine and curre...",Died


In [127]:
df.to_csv('serialized1.csv')

In [130]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)


In [132]:
dataset

Dataset({
    features: ['description', 'outcome'],
    num_rows: 13678
})